<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/%E3%80%8CGPT4Dev_ch05%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 突破時空限制–整合搜尋功能

**準備工作**

安裝必要的套件與匯入相關模組後建立用戶端物件

In [ ]:
!pip install openai
!pip install rich
import openai
from google.colab import userdata
from rich import print as pprint
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## 5-1 用搜尋網頁幫 AI 補充知識

### 使用 Google 搜尋

In [ ]:
!pip install googlesearch-python

In [ ]:
from googlesearch import search

In [ ]:
for item in search("2023 金馬獎影后"):
    print(item)

使用進階搜尋選項

In [ ]:
for item in search(
    "2023 金馬獎影后", advanced=True, num_results=3):
    print(item.title)
    print(item.description)
    print(item.url)
    print()

## 5-2 整合搜尋結果讓 AI 跟上時代

加入網頁搜尋的聊天程式

In [ ]:
def get_reply_s(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-4-1106-preview",
            messages = messages,
            stream = True
        )
        for chunk in response:
            yield chunk.choices[0].delta.content or ''
    except openai.APIError as err:
        reply = f"發生錯誤\n{err.error.message}"

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_w(sys_msg, user_msg):
    global hist
    web_res = []
    if user_msg[:3].lower() == '/w ': # /w 代表要搜尋網路
        user_msg = user_msg[3:]       # 移除指令留下實際的訊息
        content = "以下為已發生的事實：\n"
        for res in search(user_msg, advanced=True,
                          num_results=5, lang='zh-TW'):
            content += f"標題：{res.title}\n" \
                       f"摘要：{res.description}\n\n"
        content += "請依照上述事實回答以下問題：\n"
        web_res = [{"role": "user", "content": content}]
    web_res.append({"role": "user", "content": user_msg})
    reply_full = ""
    for reply in get_reply_s(         # 使用串流版的函式
        hist                          # 先提供歷史紀錄
        + web_res                     # 再提供搜尋結果及目前訊息
        + [{"role": "system", "content": sys_msg}]):
        reply_full += reply           # 記錄到目前為止收到的訊息
        yield reply                   # 傳回本次收到的片段訊息
    hist.append({"role": "user", "content": user_msg})
    hist.append({"role":"assistant", "content":reply_full})
    hist = hist[-2 * backtrace:]      # 保留最新對話

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_w(sys_msg, msg):
        print(reply, end = "")
    print('\n')
hist = []

## 5-3 使用 Google Search JSON API

### 使用 HTTP API 取得搜尋結果

In [ ]:
import requests

In [ ]:
r = requests.get(
    'https://www.googleapis.com/customsearch/v1?' \
    'key={}&' \
    'cx={}&' \
    'num={}&' \
    'q={}'.format(
        '你的 Google API 金鑰',
        '你的搜尋引擎 ID',
        2,
        '2023 金馬獎影后是誰？'
    )
)

In [ ]:
pprint(r.json())

In [ ]:
for item in r.json()['items']:
    print(item['title'])
    print(item['snippet'])
    print(item['link'])
    print()

### 使用客製模組

In [ ]:
!git clone https://github.com/codemee/customsearchapi.git \
customsearchapi

In [ ]:
# 預設會在匯入時從環境變數 GOOGLE_API_KEY 與 GOOGLE_CSE_ID
# 讀取你的 Google API 金鑰與搜尋引擎 ID,
# 如果沒有設定, 也可以直接透過模組內的變數設定：
import customsearchapi
customsearchapi.GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
customsearchapi.GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID')

In [ ]:
from customsearchapi import search

In [ ]:
for item in search("2023 金馬獎影后是誰？",
                   advanced=True,
                   num_results=3,
                   lang='lang_zh-TW'
                   ):
    print(item.url)
    print(item.title)
    print(item.description)
    print()